In [3]:
import numpy as np
import csv
import GPy
from matplotlib import pyplot as plt
import pandas as pd
import warnings
warnings.simplefilter("error", RuntimeWarning)

In [4]:
def cnt_rmse(y, y_hat):
    return np.sqrt(((y_hat.flatten()-y.flatten())**2).mean())

In [11]:
fname = '/home/chun/Desktop/gpr_testing/parkinsons_telemonitoring/parkinsons_updrs.data'


df = pd.read_csv(fname, header=0)
X_df = df.drop(df.columns[[4,5]], axis=1)
Y_df = df.iloc[:,4]

print(X_df.head(5))
Y_df.head(5)

   subject#  age  sex  test_time  Jitter(%)  Jitter(Abs)  Jitter:RAP  \
0         1   72    0     5.6431    0.00662     0.000034     0.00401   
1         1   72    0    12.6660    0.00300     0.000017     0.00132   
2         1   72    0    19.6810    0.00481     0.000025     0.00205   
3         1   72    0    25.6470    0.00528     0.000027     0.00191   
4         1   72    0    33.6420    0.00335     0.000020     0.00093   

   Jitter:PPQ5  Jitter:DDP  Shimmer  Shimmer(dB)  Shimmer:APQ3  Shimmer:APQ5  \
0      0.00317     0.01204  0.02565        0.230       0.01438       0.01309   
1      0.00150     0.00395  0.02024        0.179       0.00994       0.01072   
2      0.00208     0.00616  0.01675        0.181       0.00734       0.00844   
3      0.00264     0.00573  0.02309        0.327       0.01106       0.01265   
4      0.00130     0.00278  0.01703        0.176       0.00679       0.00929   

   Shimmer:APQ11  Shimmer:DDA       NHR     HNR     RPDE      DFA      PPE  
0        

0    28.199
1    28.447
2    28.695
3    28.905
4    29.187
Name: motor_UPDRS, dtype: float64

In [16]:
X = X_df.values
Y = np.reshape(Y_df.values, (len(X),1))
X.shape

(5875, 20)

In [33]:
kernel = GPy.kern.RBF(input_dim = 20, variance = 1., lengthscale = 1., ARD = True)
# kernel = GPy.kern.RBF(input_dim = 20, variance = 1., lengthscale = 1.)
optimize = True
# optimize = False

data_idx = np.arange(len(X))
np.random.shuffle(data_idx)

test_size = int(np.floor(X.shape[0]/10))
split_idx = [test_size*i for i in range(1,10)]
test_idx_list = np.split(data_idx, split_idx)

train_size = 0
train_size = 2000
# train_size = 1000
# train_size = 500
# train_size = 300
# train_size = 100

err_list = []
for test_idx in test_idx_list:
    all_idx = np.arange(len(X))
    tmp_train_idx = np.delete(all_idx, test_idx)
    test_X = X[test_idx]
    if train_size == 0:
        new_train_idx = tmp_train_idx
    else:
        np.random.shuffle(tmp_train_idx)
        new_train_idx = tmp_train_idx[:train_size]

    new_X = X[new_train_idx]
    print(test_X.shape)
    print(new_X.shape)
    
    # normalize y 
    Y_mean = Y - Y[new_train_idx].mean()
    test_y = Y_mean[test_idx]
    new_y = Y_mean[new_train_idx]
    
    try:
#         m = GPy.models.GPRegression(new_X, new_y, kernel, normalizer=True)
        m = GPy.models.GPRegression(new_X, new_y, kernel, normalizer=False)
    except RuntimeWarning as e:
        print(e, '0')
#         continue
    if optimize:
        try:
            m.optimize(max_iters=50, optimizer="lbfgs")
    #         m.optimize(max_iters=100)
        except RuntimeWarning as e:
            print(e, '1')
#             continue
    try:
        y_hat = m.predict(test_X)[0]
    except RuntimeWarning as e:
        print(e, '2')
#         continue
    try:
        err = cnt_rmse(y_hat, test_y)
    except RuntimeWarning as e:
        print(e, '3')
#         continue
    
    err_list.append(err)

print(sum(err_list)/len(err_list))

(587, 20)
(2000, 20)
(587, 20)
(2000, 20)
(587, 20)
(2000, 20)
(587, 20)
(2000, 20)
overflow encountered in expm1 1
(587, 20)
(2000, 20)
overflow encountered in expm1 1
(587, 20)
(2000, 20)
overflow encountered in expm1 1
(587, 20)
(2000, 20)
overflow encountered in expm1 1
(587, 20)
(2000, 20)
overflow encountered in expm1 1
(587, 20)
(2000, 20)
overflow encountered in expm1 1
(592, 20)
(2000, 20)
overflow encountered in expm1 1
0.19753743344825084
